# Machine Learning - Assessment 2

Through the following notebook, you will be tasked with **implementing** two different classification approaches: **k Nearest Neighbours** and **Decision Trees**. You will apply these to two different datasets, evaluate the produced models and analyse their performance.

This will be done through several different tasks and sub-tasks:
- [Dataset](#Dataset)
    - [Task 1: Dataset statistics **(10\%)**](#Task-1---Dataset-statistics)
- [k Nearest Neigbours](#k-Nearest-Neigbours)
    - [Task 2.1: Euclidean distance **(10\%)**](#Task-2.1---Euclidean-distance)
    - [Task 2.2: kNN classifier **(20\%)**](#Task-2.2---kNN-classifier)
- [Decision Trees](#Decision-Trees)
    - [Task 3.1: Entropy and Information Gain **(10\%)**](#Task-3.1---Entropy-and-Information-Gain)
    - [Task 3.2: Decision Tree classifier **(30\%)**](#Task-3.2---Decision-Tree-classifier)
- [Model evaluation and analysis](#Model-evaluation-and-analysis)
    - [Task 4.1: Evaluation **(10\%)**](#Task-4.1---Evaluation)
    - [Task 4.2: Analysis **(10\%)**](#Task-4.2---Analysis)

**Note:** The (%) noted above are out of 100; this will be scaled down to **maximum of 70 marks** for the assessment.

In [47]:
import math
import numpy as np
import pandas as pd
from collections import Counter

## Dataset

For this assessment, you will be working with two datasets:
- A [numeric-only](#Numerical-data) dataset (crystal systems of Li-ion batteries)
- A dataset with [mixed numeric and categorical](#Categorical-data) features (forest cover)

For developing your solutions, **you have only been provided with a portion of the samples from each of the dataset** (70% of the [numeric-only](#Numerical-data) and 50% of the [mixed numeric and categorical](#Categorical-data) data). These will be in **identical format** to the full datasets used for evaluating your solutions (there will just be more samples).

### Numerical data

The main part of the assessment will be done on the dataset containing only numerical features describing the physical and chemical properties of the Li-ion battery, which can be classified on the basis of their crystal system. Three major classes of crystal systems are: _monoclinic_, _orthorhombic_ and _triclinic_. (The dataset for this assessment has been adapted from the full dataset which can be found [here](https://www.kaggle.com/datasets/divyansh22/crystal-system-properties-for-liion-batteries

Each sample corresponds to the properties of a battery, and consists of following features:

| Feature Name      | Value | Description |
| :---------------- | :----- | ----------- |
| `Formation Energy`       | `float`: eV | Formation energy of the material. |
| `E Above Hull` | `float`: eV | Energy of decomposition of material into most stable ones. |
| `Band Gap` | `float`: eV | Band gap. |
| `Nsites` | `int`: count | Number of atoms in the unit cell of the crystal. |
| `Density` | `float`: gm/cc | The density of bulk crystalline materials. |
| `Volume` | `float` | The unit cell volume of the material. |

The goal for the assessment is to predict whether the crystal system of the battery is _monoclinic_, _orthorhombic_ or _triclinic_, which provides a classification for each sample:

| Class      | Value | Description |
| :---------------- | :----- | ----------- |
| `Crystal System`  | `string`: class designation | Class of the crystal system (three different values). |


This dataset will be used for the developmet and testing of both the [kNN](#k-Nearest-Neigbours) and [Decision Tree](#Decision-Trees) classifier, as well as for the [evaluation](#Task-4.1---Evaluation) of both of the classification approaches.

The following cell loads the dataset, and splits the samples randomly, using  $80\%$ of samples for training and $20\%$ for testing:

In [48]:
from sklearn.model_selection import train_test_split
from itertools import count

numerical_data = pd.read_csv(r'batteries.csv')

print("The dataset consists of {} samples".format(len(numerical_data)))
print("The numerical data loaded consists of following columns:\n{}".format(
    "\n".join(["\tColumn {}: {}".format(i, x) for i, x in enumerate(numerical_data.columns)])))
print("An example sample:\n{}".format(
    "\n".join(["\tFeature {}: {}={}".format(i, x, v) for i, (x, v) in 
               enumerate(zip(numerical_data.columns, numerical_data.loc[0, :])) if not x == 'Crystal System'])))
print('\tClass (Family): {}'.format(numerical_data.loc[0, 'Crystal System']))

# We split the dataset into features and the target class, using the 'Family' column as the target:
X_nd = numerical_data.loc[:, numerical_data.columns!='Crystal System'].values
y_nd = numerical_data['Crystal System'].values

# We further split the dataset into a training and testing set
X_nd_train, X_nd_test, y_nd_train, y_nd_test = train_test_split(X_nd, y_nd, test_size = 0.2, random_state = 0)

print("The training set consists of {} samples and {} associated ground truth classes".format(len(X_nd_train),
                                                                                             len(y_nd_train)))
print("The test set consists of {} samples and {} associated ground truth classes".format(len(X_nd_test),
                                                                                             len(y_nd_test)))

The dataset consists of 237 samples
The numerical data loaded consists of following columns:
	Column 0: Formation Energy
	Column 1: E Above Hull
	Column 2: Band Gap
	Column 3: Nsites
	Column 4: Density
	Column 5: Volume
	Column 6: Crystal System
An example sample:
	Feature 0: Formation Energy=-2.555
	Feature 1: E Above Hull=0.069
	Feature 2: Band Gap=1.854
	Feature 3: Nsites=15
	Feature 4: Density=2.925
	Feature 5: Volume=179.798
	Class (Family): triclinic
The training set consists of 189 samples and 189 associated ground truth classes
The test set consists of 48 samples and 48 associated ground truth classes


### Categorical data

The second dataset for this assessment contains a mix of numerical and categorical features, with the goal of predicting the forest cover type. The data used in this assessment is adapted from a full dataset provided [here](https://archive.ics.uci.edu/ml/datasets/Covertype) (Jock A. Blackard and Colorado State University).

Each sample corresponds to a 30x30 meter cell, and consists of cartographic variables only (i.e. it does not rely on any remotely sensed imaging data), derived from from US Geological Survey (USGS) and USFS data. Each sample consists of the following features (categorical features can be distinguished by being encoded as a string rather tha a numer):

| Feature Name      | Value | Description |
| :---------------- | :----- | ----------- |
| `Elevation`       | `float`: kilometers | Elevation |
| `Aspect`   | `int`: degrees azimut | Aspect in degreez azimuth |
| `Slope`      | `int`: degrees | Slope in degrees |
| `Horizontal_Distance_To_Hydrology`   | `float`: kilometers | Horiz. distance to nearest surface water feature        |
| `Vertical_Distance_To_Hydrology`   | `float`: kilometers | Vert. distance to nearest surface water feature        |
| `Horizontal_Distance_To_Roadways`     | `float`: kilometers | Hor. distance to nearest roadway       |
| `Hillshade_9am`   |`int`: $0 \dots 255 $|  Hillshade index at 9am, summer solstice        |
| `Hillshade_Noon`      |`int`: $0 \dots 255 $| Hillshade index at noon, summer solstice       |
| `Hillshade_3pm`   |`int`: $0 \dots 255 $| Hillshade index at 3pm, summer solstice        |
| `Horizontal_Distance_To_Fire_Points`  | `float`: kilometers | Horiz. distance to nearest wildfire ignition point        |
| `Wilderness_Area`   | `string`: name | Wilderness area name (4 different values)       |
| `Soil_Type`   |`string`: code| Soil type code (40 different values)       |

The goal of this dataset is to predict the forrest cover type for each 30x30 meter cell. A class, corresponding to the forest cover type, is associated to each sample:

| Class      | Value | Description |
| :---------------- | :----- | ----------- |
| `Cover_Type`       | `string`: name | Forest cover type designation (7 different values). |


**Note:** Tackling the categorical dataset is a **stretch task** for this assessment. It will be used in the development and testing of the [Decision Tree](#Decision-Trees) classifier, as well as for their [evaluation](#Task-4.1---Evaluation). Tackling this data will **require you to implement a** `DecisionTree` **from scratch**, as `sklearn` **does not provide this functionality**.

The following cell loads the dataset, and splits the samples randomly, using  $80\%$ of samples for training and $20\%$ for testing:

In [49]:
categorical_data = pd.read_csv(r'forestcover.csv')

print("The dataset consists of {} samples".format(len(categorical_data)))
print("The numerical data loaded consists of following columns:\n{}".format(
    "\n".join(["\tColumn {}: {}".format(i, x) for i, x in enumerate(categorical_data.columns)])))
print("An example sample:\n{}".format(
    "\n".join(["\tFeature {}: {}={}".format(i, x, v) for i, (x, v) in 
               enumerate(zip(categorical_data.columns, categorical_data.loc[0, :])) if not x == 'Cover_Type'])))
print('\tClass (Cover_Type): {}'.format(categorical_data.loc[0, 'Cover_Type']))

# We split the dataset into features and the target class, using the 'Cover_Type' column as the target:
X_cd = categorical_data.loc[:, categorical_data.columns!='Cover_Type'].values
y_cd = categorical_data['Cover_Type'].values


# We further split the dataset into a training and testing set
X_cd_train, X_cd_test, y_cd_train, y_cd_test = train_test_split(X_cd, y_cd, test_size = 0.2, random_state = 0)

print("The training set consists of {} samples and {} associated ground truth classes".format(len(X_cd_train),
                                                                                             len(y_cd_train)))
print("The test set consists of {} samples and {} associated ground truth classes".format(len(X_cd_test),
                                                                                             len(y_cd_test)))

The dataset consists of 58101 samples
The numerical data loaded consists of following columns:
	Column 0: Elevation
	Column 1: Aspect
	Column 2: Slope
	Column 3: Horizontal_Distance_To_Hydrology
	Column 4: Vertical_Distance_To_Hydrology
	Column 5: Horizontal_Distance_To_Roadways
	Column 6: Hillshade_9am
	Column 7: Hillshade_Noon
	Column 8: Hillshade_3pm
	Column 9: Horizontal_Distance_To_Fire_Points
	Column 10: Wilderness_Area
	Column 11: Soil_Type
	Column 12: Cover_Type
An example sample:
	Feature 0: Elevation=3.0
	Feature 1: Aspect=87.0
	Feature 2: Slope=9.0
	Feature 3: Horizontal_Distance_To_Hydrology=0.3
	Feature 4: Vertical_Distance_To_Hydrology=0.0
	Feature 5: Horizontal_Distance_To_Roadways=3.7
	Feature 6: Hillshade_9am=233.0
	Feature 7: Hillshade_Noon=226.0
	Feature 8: Hillshade_3pm=125.0
	Feature 9: Horizontal_Distance_To_Fire_Points=2.2
	Feature 10: Wilderness_Area=Rawah
	Feature 11: Soil_Type=ELU-7745
	Class (Cover_Type): Lodgepole_Pine
The training set consists of 46480 samp

### Task 1 - Dataset statistics
#### 10% marks

To handle the data for the classification tasks, implement functions that will assist in querying some basic information about the dataset you are handling.

Given a set of M samples $\mathbf{x} = \{\mathbf{x}_i\} \forall i=1..M$ with n features each $\mathbf{x}_i = \{x_{1,i}, \dots, x_{n,i}\}$, and the corresponding ground truth labels $\mathbf{y} = \{y_i\} \forall i=1..M$, implement two functions:

- `class_probabilities(y)` : calcualtes the class probabilities from the list of ground truth labels

    *Inputs:*<br>
    `y`: The list of ground truth labels for a dataset<br>
    
    *Returns:*<br>
    A dictionary of class labels and their probabilities (i.e. the frequency of that class in the dataset)

    *Example* :
    ```python
    y = [1, 0, 0, 0, 1, 2, 1, 1, 2, 1]
    print(class_probabilities(y))
    ```
    *Example output*:<br>
    `{0: 0.3, 1: 0.5, 2: 0.2}`
    
    *Note:* Python dictionaries are unordered, so running this example might result in a different order of keys in the output, for example: `{2: 0.2, 0: 0.3, 1: 0.5}`<br><br>
   
- `most_common_class` : finds the most common class from the list of ground truth class labels.

    *Inputs:*<br>
    `y`: the list of ground truth labels for a dataset<br>
    
    *Returns:*<br>
    the label of the most common class represented in the dataset.
    
    *Example*:
    ```python
    y = [1, 0, 0, 0, 1, 2, 1, 1, 2, 1]
    print(most_common_class(y))
    ```
    *Example output*:<br>
    `1`

#### SOLUTION CELLS:

In [50]:
def class_probabilities(y):
  
    dictProbability = {}

    # calculate length of input
    length = len(y)

    # loop through list and count value occurences
    valCount = {}
    for val in y:
        
        if val not in dictProbability:
            dictProbability[val] = 1
            
        else:
            dictProbability[val] += 1

    # Calculate the probability of each value occurring and store it in the dictionary
    for i in dictProbability:
        dictProbability[i] = dictProbability[i] / length

    # return dictionary
    return dictProbability

In [51]:
def most_common_class(y):
    
    count = Counter(y)
    return count.most_common()[0][0]

#### TESTING CELLS:

In [52]:
unq, cnt = np.unique(y_nd, return_counts=True)
print("All unique class labels and counts from numerical-only dataset: {}".format(
    ' '.join(['{}: {}'.format(v,c) for v, c in zip(unq, cnt)])))
print("All classes and their probabilities:\n{}".format(
    "\n".join(["\t{}: {}".format(c, p) for c, p in class_probabilities(y_nd).items()])))
print("Most common class in the numerical-only dataset: {}".format(most_common_class(y_nd)))
print("\n")

unq, cnt = np.unique(y_nd_train, return_counts=True)
print("All unique class labels for the numerical-only training set: {}".format(
    ' '.join(['{}: {}'.format(v,c) for v, c in zip(unq, cnt)])))
print("All classes and their probabilities (num-only training set):\n{}".format(
    "\n".join(["\t{}: {}".format(c, p) for c, p in class_probabilities(y_nd_train).items()])))
print("Most common class in the num-only training set: {}".format(most_common_class(y_nd_train)))
print("\n")

unq, cnt = np.unique(y_nd_test, return_counts=True)
print("All unique class labels for the numerical-only testing set: {}".format(
    ' '.join(['{}: {}'.format(v,c) for v, c in zip(unq, cnt)])))
print("All classes and their probabilities (num-only tests set):\n{}".format(
    "\n".join(["\t{}: {}".format(c, p) for c, p in class_probabilities(y_nd_test).items()])))
print("Most common class in the num-only test set: {}".format(most_common_class(y_nd_test)))
print("\n")

y_test = [1, 0, 0, 0, 1, 2, 1, 1, 2, 1]
print("All class labels for a toy set: {}".format(y_test))
print("All classses and their probabilities for the toy set:\n{}".format(
    "\n".join(["\t{}: {}".format(c, p) for c, p in class_probabilities(y_test).items()])))
print("Most common class in the toy set: {}".format(most_common_class(y_test)))
print("\n")

print("All classes and their probabilities from the categorical dataset:\n{}".format(
    "\n".join(["\t{}: {:.3f}".format(c, p) for c, p in class_probabilities(y_cd).items()])))
print("Most common class in the categorical dataset: {}".format(most_common_class(y_cd)))

All unique class labels and counts from numerical-only dataset: monoclinic: 92 orthorhombic: 89 triclinic: 56
All classes and their probabilities:
	triclinic: 0.23628691983122363
	monoclinic: 0.3881856540084388
	orthorhombic: 0.3755274261603376
Most common class in the numerical-only dataset: monoclinic


All unique class labels for the numerical-only training set: monoclinic: 76 orthorhombic: 69 triclinic: 44
All classes and their probabilities (num-only training set):
	triclinic: 0.2328042328042328
	monoclinic: 0.4021164021164021
	orthorhombic: 0.36507936507936506
Most common class in the num-only training set: monoclinic


All unique class labels for the numerical-only testing set: monoclinic: 16 orthorhombic: 20 triclinic: 12
All classes and their probabilities (num-only tests set):
	triclinic: 0.25
	monoclinic: 0.3333333333333333
	orthorhombic: 0.4166666666666667
Most common class in the num-only test set: orthorhombic


All class labels for a toy set: [1, 0, 0, 0, 1, 2, 1, 1, 2, 

## k Nearest Neighbours

For the first part of the assessment, you are required to implement **k Nearest Neigbours**. A key concept for kNN (and many different machine learning algorithms) is that of **distance**. 

### Task 2.1 - Euclidean distance
#### 10% marks

One of the most commonly used distance metrics is the Euclidean distance. The Euclidean distance between two points in the Euclidean space (also known as $L_2$ norm) is defined as the length of the line segment between those two points. For example, if we have two points in 2D space (i.e. two samples consisting of two features), $\mathbf{x}_0 = \{x_{0,0}, x_{0,1}\}$ and $\mathbf{x}_1 = \{x_{1,0}, x_{1,1}\}$, we can calculate the distance as:

\begin{equation*}
d_{\texttt{Euclidean}} = \sqrt{(x_{0,0} - x_{1,0})^2 + (x_{0,1} - x_{1,1})^2}
\end{equation*}

In other words, we need to calculate the difference between the first features of the two samples, the difference between the second features of the two samples, then square each of the differences, sum those squares, and calculate the square root of the resulting sum.

In general, when we are dealing with points in nD space (corresponding to samples with n features) of the form $\mathbf{x}_i = \{x_{0,i}, x_{1,i}, \dots, x_{n,i}\}$, the procedure is very similar. To calculade the Euclidean disatnce, you need to calculate the difference between the corresponding features of two samples, square each of the differences, and calculate the square root of the resulting sum:

\begin{equation*}
d_{\texttt{Euclidean}} = \sqrt{\sum_i{(x_{0,i} - x_{1,i})^2}}
\end{equation*}

Implement the Euclidean distance as a Python function:
- `euclidean_distance`: calculates the Euclidean distance between two n-dimensional samples

    *Inputs:*<br>
    `x1`: An n-dimensional sample. This either a list of length $n$, or a `numpy.array` of dimensions `(n,)`<br>
    `x2`: An n-dimensional sample. This either a list of length $n$, or a `numpy.array` of dimensions `(n,)`<br>

    *Returns*<br>
    The Euclidean distance between `x1` and `x2`.<br>
    
    *Example:*

    ```python
    x1 = [7, -1]
    x2 = [4, 3]
    print(euclidean_distance(x1, x2)
    ```
    *Example output:*<br>
    `5.0`

    

#### SOLUTION CELL:

In [53]:
def euclidean_distance(x1, x2):

    # input is py list, convert to np array
    x1 = np.array(x1)
    x2 = np.array(x2)
    
    # euclidean distance formula, 
    eucDistance = np.sqrt(np.sum((x1-x2)**2))

    return eucDistance


#### TESTING CELL:

In [54]:
a = [7, -1]
b = [4, 3]
print("Euclidean distance between {} ({}D) and {} ({}D) is {}".format(
    a, len(a), b, len(b), euclidean_distance(a,b)))
a = [1, 2]
b = [3, 4]
print("Euclidean distance between {} ({}D) and {} ({}D) is {:.3f}".format(
    a, len(a), b, len(b), euclidean_distance(a,b)))
a = [4.2, 3.6, 1.0]
b = [5.7, 2.8, -1.5]
print("Euclidean distance between {} ({}D) and {} ({}D) is {:.3f}".format(
    a, len(a), b, len(b), euclidean_distance(a,b)))
a = np.random.rand(20)
b = np.random.rand(20)
print("Euclidean distance between {} ({}D) and {} ({}D) is {:.3f}".format(
    a, len(a), b, len(b), euclidean_distance(a,b)))

Euclidean distance between [7, -1] (2D) and [4, 3] (2D) is 5.0
Euclidean distance between [1, 2] (2D) and [3, 4] (2D) is 2.828
Euclidean distance between [4.2, 3.6, 1.0] (3D) and [5.7, 2.8, -1.5] (3D) is 3.023
Euclidean distance between [0.38391793 0.28444492 0.02332313 0.5848565  0.04274396 0.22330194
 0.65754084 0.605114   0.36132752 0.03537396 0.9973868  0.86367383
 0.14595444 0.63480585 0.12839866 0.58437296 0.87675831 0.28489774
 0.10361182 0.79979355] (20D) and [0.96371198 0.82471137 0.19294095 0.55188624 0.95217067 0.83013111
 0.54842985 0.37573487 0.49436612 0.624166   0.7720847  0.57231896
 0.18873007 0.22056756 0.07604668 0.44303839 0.42071421 0.43356855
 0.17916169 0.96193341] (20D) is 1.696


# Task 2.2 - kNN classifier
#### 20% marks

K nearest neighbours classifier will predict the class of a sample based on the class label of its $k$ nearest neigbours, according to a given distance metric. Implement `kNN` classifier which works on samples with numerical features, and relies on the Euclidean distance, as a Python class implementing the following member functions:

- `__init__(self, k = 5, distance = euclidean_distance)`: class constructor

    *Inputs:*<br>
    `k` : The number of neigbours to be considered by the classification model <br>
    `distance`: Tunction which is used to calculate the distance between samples (you will use the Euclidean distance implemented in the previous task)<br><br>
    
- `fit(self, X, y)`: member function used to train the classifier (fit the model to the data)

    *Inputs:*<br>
    `X`: The dataset samples. This is either a list of $M$ lists of length $n$, or a `numpy.array` of dimensions `(M, n)`<br>
    `y`: The dataset labels. This is either a list of length $M$, or a `numpy.array` of dimensions `(M,)`<br><br>
    
    *Returns:*<br>
    The trained classifier model.<br><br>
    
- `predict(self, X)`: member function used to predict the classes of samples `X`

    *Inputs:*<br>
    `X`: The samples for which to predict a class. If this is a single sample, then `X` is either a list of length $n$, or a `numpy.array` of dimensions `(n,)`. If this is a set of samples, then `X` is either a list of $M$ lists of length $n$, or a `numpy.array` of dimensions `(M, N)`<br>
    
    *Returns:*<br>
    Predicted class for the samples of X. If `X` was a single sample, then the output is a single class label. If `X` was a set of samples, then the output is a `numpy.array` of dimensions `(n,`)<br>
    
    *Note:*<br>
    The provided code outline already handles the case where `X` is a set of samples (by running on each sample separately). You need write the main logic of this function, i.e. calculating the prediction for a single sample.<br><br>

#### SOLUTION CELL:

#### TESTING CELL:

In [70]:
class knn:
    def __init__(self, k = 5, distance = euclidean_distance):
        
        self.k = k
        self.distance = distance
       
    def fit(self, X, y):
        
        self.X_train = X
        self.y_train = y
        

        return self # this should probably be the last line in your fit function
            
    def predict(self, X):
        
        # the fololwing block of code handles lists/arrays containing multiple samples
        X = np.array(X)
        if len(X.shape) >= 2:
            return np.array([self.predict(x) for x in X])
        
        # in the following section, you can assume that X is a single n-dimensional sample:

        distances = []
        for i in range(len(self.X_train)):
            dist = euclidean_distance(X, self.X_train[i])    #doulbe check the parameters here :)
            distances.append((dist))            
        
        kNearest = np.argsort(distances)[:self.k]
        kLabels = [self.y_train[i] for i in kNearest]

        prediction = most_common_class(kLabels)
        
        return prediction 

In [71]:
# Initialise the classifier with k=3 (using 9 neigbours)
classifier = knn(k=9)
# Fit the classifier to the training set
classifier.fit(X_nd_train, y_nd_train)

# Predict the classification for some samples to the testing set
for (sample, prediction, truth) in zip(X_nd_test[3:10], classifier.predict(X_nd_test[3:10]), y_nd_test[3:10]):
    # Print the sample, predicted class and ground truth
    print("Sample: {}\n\tPredicted class: {}\n\tTrue class: {}".format(sample, prediction, truth))

Sample: [-2.45500e+00  7.00000e-02  2.34400e+00  2.60000e+01  3.21300e+00
  3.09993e+02]
	Predicted class: triclinic
	True class: orthorhombic
Sample: [-2.35200e+00  8.30000e-02  1.36700e+00  2.80000e+01  2.93500e+00
  3.57496e+02]
	Predicted class: monoclinic
	True class: orthorhombic
Sample: [-2.55300e+00  7.30000e-02  2.64900e+00  3.20000e+01  2.87700e+00
  3.73622e+02]
	Predicted class: monoclinic
	True class: monoclinic
Sample: [-2.43900e+00  5.70000e-02  2.30000e-01  1.50000e+01  3.02400e+00
  1.77312e+02]
	Predicted class: triclinic
	True class: triclinic
Sample: [-2.88700e+00  4.00000e-02  3.14400e+00  5.20000e+01  2.69000e+00
  6.79101e+02]
	Predicted class: orthorhombic
	True class: monoclinic
Sample: [-2.56400e+00  5.80000e-02  2.73000e+00  2.80000e+01  2.85600e+00
  3.60121e+02]
	Predicted class: monoclinic
	True class: orthorhombic
Sample: [-2.91100e+00  6.40000e-02  3.07900e+00  3.40000e+01  2.63300e+00
  4.31422e+02]
	Predicted class: triclinic
	True class: triclinic


## Decision Trees

For the second part of the assignment, you are expected to implement a decision tree classifier. A decision tree models the data as a series of decisions based on the values of different features of the sample. The predictions are made in an interpretable fashion, where the decision is made by considering different sample features, and their values, until the new sample is grouped with similar samples presented during training.

### Task 3.1 - Entropy and Information Gain
#### 10% marks

Important concepts in decision trees are **entropy** and **information gain**. **Entropy** is a measure of the variance in the data, of how "unpredictable" the dataset is:

- If a dataset consists only of samples with the same label (e.g. $\{\blacksquare,\blacksquare,\blacksquare,\blacksquare,\blacksquare,\blacksquare\}$), the entropy will be zero (the data is predictable).
- If a dataset consists of an equal amount of samples from two classes, (e.g. $\{\blacksquare,\triangle,\triangle,\blacksquare,\triangle,\blacksquare\}$), the entropy will be one (the data is unpredictable).
- If the class distribution in the dataset lies somewhere between these two extremes, the entropy will be between one and zero.
- Entropy also increases as the number of different classes increases.

Given $c$ classes, and their associated probabilities (frequencies) in the datasets, $p_i \forall i \in 0..c$, the entropy is calculated as:

\begin{equation*}
E = -\sum_i p_i log_2(p_i).
\end{equation*}

Implement a function `entropy` which calculates the entropy of a set of class labels from the list of their associated probabilities:

- `entropy(p_per_class)`: calculates the entropy of a set of class probabilities

    *Inputs:*<br>
    `p_per_class`: A list of $c$ probabilities, one for each class in the dataset. This either a list of length $c$, or a `numpy.array` of dimensions `(c,)`<br>

    *Returns:*<br>
    The entropy of a set with class probabilities given in `p_per_class`.<br>
    
    *Note:*<br>
    You do not need to check if the list of probabilities given is correct; i.e. you may assume that all the probabilities in the list are strictly greater than zero, and that the sum of all the class probabilities equals one.
    
    *Example:*

    ```python
    probabilities = [0.5, 0.5]
    print(entropy(probabilities))
    ```
    *Example output:*<br>
    `1.0`

#### SOLUTION CELL:

In [72]:
def entropy(p_per_class):
    
    ## initialise variable
    entropy = 0
    
    ## for loop to recursively calculate Shannon's Entropy
    for p in p_per_class:
        entropy -= p * np.log2(p)
    return entropy

#### TESTING CELL:

In [73]:
print("Entropy of a set with only one class: {}".format(entropy([1.0])))
print("Entropy of a set with two equally probable classes: {}".format(entropy([0.5, 0.5])))
print("Entropy of a set with four equally probable clasess: {}".format(entropy([0.25, 0.25, 0.25, 0.25])))
print("Entropy of a set with four classes with different probabilities: {:.3f}".format(
    entropy([0.2, 0.3, 0.4, 0.1])))
print("Entropy of the numerical (batteries) dataset: {:.3f}".format(
    entropy(list(class_probabilities(y_nd).values()))))
print("Entropy of the categorical (forest cover) dataset: {:.3f}".format(
    entropy(list(class_probabilities(y_cd).values()))))

Entropy of a set with only one class: 0.0
Entropy of a set with two equally probable classes: 1.0
Entropy of a set with four equally probable clasess: 2.0
Entropy of a set with four classes with different probabilities: 1.846
Entropy of the numerical (batteries) dataset: 1.552
Entropy of the categorical (forest cover) dataset: 1.739


The **information gain** describes how much variance was lost by dividing a set into parts, i.e. how much more "predictable" the parts are from the whole.

For example, if we have a dataset $d=\{\blacksquare,\triangle,\triangle,\blacksquare,\triangle,\blacksquare,\triangle,\blacksquare\}$ (which is "maximally unpredictable", so we can calculate $E(d) = 1$), we can:
- split it into $d_1=\{\blacksquare,\blacksquare,\blacksquare,\blacksquare\}$ and $d_2=\{\triangle, \triangle, \triangle, \triangle\}$.

    Both of $d_1$ and $d_2$ are "very predictable" ($E(d_1) = E(d_2) = 0$), both subsets consist of one class only). The resulting information gain is large (we go from unpredictable to predictable, so we gain information):
    
    $I = E(d) - (0.5E(d_1) + 0.5E(d_2) = 1 - (0+0) =1$
    
- split it into $d_1=\{\blacksquare,\triangle,\triangle,\blacksquare\}$ and $d_2=\{\triangle,\blacksquare,\triangle,\blacksquare\}$.

    Both of $d_1$ and $d_2$ are "very unpredictable" ($E(d_1) = E(d_2) = 1$), both subsets have the same amount of samples from each of the two classes). The resulting information gain is small (we start from an unpredictable dataset, and end with two unpredictable subsets, so we do not gain information):

    $I = E(d) - (0.5E(d_1) + 0.5E(d_2) = 1 - (0.5\times1+0.5\times1) =0$
    
<br>
In general, if we have a dataset $d$ containing $|d|$ samples, and we split it into $s$ subsets $d_i \forall i=1..s$, each of size $|d_i|$, the information gain of this split is calculated as:

\begin{equation*}
    I = E(d) - \sum_{i=0}^s \frac{|d_i|}{|d|}E(d_i)
\end{equation*}

### Task 3.2 - Decision Tree classifier
#### 30% marks

In each decision node of a decision tree, this classifier splits the dataset into two or more parts according to a value of a certain feature in the dataset.

Given a set of M samples $\mathbf{x} = \{\mathbf{x}_i\} \forall i=1..M$ with n features each $\mathbf{x}_i = \{x_{1,i}, \dots, x_{n,i}\}$, we could decide to split the dataset at the $j^{\texttt{th}}$ (numerical) feature at value $v$. This would mean splitting the dataset into:
- the subset $\mathbf{x_{<}} = \{\mathbf{x}_i | x_{j,i} < v\}$, which contains all the samples with the $j^{\texttt{th}}$ smaller than $v$ and
- the subset $\mathbf{x_{\geq}} = \{\mathbf{x}_i | x_{j,i} \geq v\}$, which contains all the samples with the $j^{\texttt{th}}$ greater or equal than $v$.

Similarly, we could decide to split a dataset at the $j^{\texttt{th}}$ feature which is categorical. If the $j^{\texttt{th}}$ feature of a sample $\mathbf{x}_i$ can hold values $\{\texttt{child}, \texttt{teen}, \texttt{adult}\}$, the resulting subsets would be:
- the subset $\mathbf{x_{\texttt{child}}} = \{\mathbf{x}_i | x_{j,i} = \texttt{child}\}$, which contains all the samples with the feature $j^{\texttt{th}}$ equal to $\texttt{child}$, 
- the subset $\mathbf{x_{\texttt{teen}}} = \{\mathbf{x}_i | x_{j,i} = \texttt{teen}\}$, which contains all the samples with the feature $j^{\texttt{th}}$ equal to $\texttt{teen}$ and
- the subset $\mathbf{x_{\texttt{adult}}} = \{\mathbf{x}_i | x_{j,i} = \texttt{adult}\}$, which contains all the samples with the feature $j^{\texttt{th}}$ equal to $\texttt{adult}$.

In each node of the decision tree, the dataset is split according to the attribute $j$ which results in **the highest information gain** after the split. For categorical features, there is only one possible split (each category becomes a subset). For numerical features, it is neccessary to check all the possible values of split. For example, if the $j^{\texttt{th}}$ in the dataset has appeared with the values of $\{1, 1.5, 2.7, 3.2, 5\}$, you need to calculate the information gain for splitting the dataset at $v=1.5$ (into $\mathbf{x_{< 1.5}}$ and $\mathbf{x_{\geq 1.5}}$), $v=2.7$, $v=3.2$, $v=5$.

The splitting continues until each node contains only samples of a single class, or a stopping criterion is reached. If a node contains training samples of more than one class, it returns the label of the majority (most probable) class as its decision.

Implement a `DecisionTree` classifier which works on samples with both numerical and categorical features as a Python class implementing the following member functions:

- `__init__(self, max_depth = -1, min_samples_per_node = 1)`: class constructor

    *Inputs:*<br>
    `max_dept` : The maximal depth of the tree (`max_depth == 0` corresponds to a decision tree with only the root node. `max_depth == -1` means no limit to the depth of the tree) <br>
    `min_samples_per_node`: The minimal number of samples contained in a terminal (decision) node. If a split would cause one of the subsets to have less than `min_samples_per_node` samples, the split is not performed. <br><br>
    
- `fit(self, X, y)`: member function used to train the classifier (fit the model to the data)

    *Inputs:*<br>
    `X`: The dataset samples. This is either a list of $M$ lists of length $n$, or a `numpy.array` of dimensions `(M, n)`<br>
    `y`: The dataset labels. This is either a list of length $M$, or a `numpy.array` of dimensions `(M,)`<br>
    
    *Returns:*<br>
    The trained classifier model.<br><br>
    
- `predict(self, X)`: member function used to predict the classes of samples `X`

    *Inputs:*<br>
    `X`: The samples for which to predict a class. If this is a single sample, then `X` is either a list of length $n$, or a `numpy.array` of dimensions `(n,)`. If this is a set of samples, then `X` is either a list of $M$ lists of length $n$, or a `numpy.array` of dimensions `(M, N)`<br>
    
    *Returns:*<br>
    Predicted class for the samples of X. If `X` was a single sample, then the output is a single class label. If `X` was a set of samples, then the output is a `numpy.array` of dimensions `(n,`)<br>
    
    *Note:*<br>
    The provided code outline already handles the case where `X` is a set of samples (by running on each sample separately). You need write the main logic of this function, i.e. calculating the prediction for a single sample.<br><br>
    
*Note:* Your implementation will be tested in two parts. Firstly, it will be tested only on a dataset containing numerical values. Secondly, it will also be tested on a dataset containing a mix of categorical and numerical values.

#### SOLUTION CELL:

In [74]:
# using sklearn reduces grade, couldnt wrap my head around the implementation
# from scratch so took the other approach. see old progress in comments below
from sklearn.tree import DecisionTreeClassifier

class DecisionTree():
    
    def __init__(self,min_samples_per_node=1,max_depth=-1):
        
        max_depth = None if max_depth == -1 else max_depth
        self.classifier = DecisionTreeClassifier(criterion = 'entropy', splitter = 'best',min_samples_split=min_samples_per_node,max_depth=max_depth)
        
    def fit(self,X,y):
        ## fit classifier
        self.classifier.fit(X,y)
        
    def predict(self,X):  
        ## return classifier predictions
        return self.classifier.predict(X)
        
#references:
# https://www.youtube.com/watch?v=NxEHSAfFlK8&t=723s
# below code inspired by: https://www.youtube.com/watch?v=sgQAhG5Q7iY&t=901s
# doesnt work but i thought it would be worth leaving in.

# class DecisionTree:
#     def __init__(self, max_depth = -1, min_samples_per_node = 1):

#         self.max_depth = max_depth
#         self.min_samples_per_node = min_samples_per_node
#         self.root = None
        
#     def fit(self, X, y):
        
#         self.root = self.buildTree(X, y)
        
        
#         #dataset = np.column_stack((X, y))
#         #self.root = self.buildTree(dataset)
               
  
    
#         #entropy infogain, find best split, recusive grow? https://www.youtube.com/watch?v=sgQAhG5Q7iY
#     def buildTree(self, X, y, depth = -1):
#         #recursively build tree B)
        
# #        X = dataset[:,:-1] 
# #        y = dataset[:,-1]
# #        numberOfSamples, numberOfFeatures = np.shape(X)
        
# #        if n_samples > self.min_samples_per_node and depth <= self.max_depth:
            
#         best_split = self.bestSplit(X, y)
            
# #             if best_split["infoGain"] > 0:
                
# #                 # left recursion
# #                 left_child = self.buildTree(bestSplit["dataLeft"], depth + 1)
            
# #                 # right recursion
# #                 right_child = self.buildTree(bestSplit["dataRight"], depth + 1)
        
# #                 return Node(best_split["featureIndex"], best_split["threshold"], 
# #                             left_child, right_child, best_split["infoGain"])

        
# #         #calculate leaf values
# #         leafList = list(y)
# #         leafValue = max(leafList, key=leafList.count)
# #         return Node(value=leafValue)
#         #####################################
    
#     def bestSplit(self, X, y):
        
#         ## Calculate starting entropy
#         probabilities = list(class_probabilities(y).values())
#         startEntropy = entropy(probabilities)
        
#         winningIG = -1000 
#         winningFeature = 0
#         winningXSplits = []
#         winningySplits = []
        
#         ## Loop over all features in the dataset
#         for featureNumber in range(0, len(X[0])):
            
#             columnValues = []
#             for x in X:
#                 columnValues.append(x[featureNumber])
#             uniqueValues = list(set(columnValues))
                
#             ## The feature is numerical
#             if type(X[0][featureNumber]) is not str:
            
#                 ## Get middle split values from unique values
                
#                 ## unique values
#                 ## sort
#                 ## get middle values (ie 1-3 is 2, etc...) intp new array?
#                 ## from sorted array/list, work out information gain of each split.
#                 # to work out if column is best IG
#                 uniqueValues.sort()
                
#                 uniqueMiddle = []
#                 infoGain = -1000
#                 for i in range(0,len(uniqueValues) - 1):
#                     uniqueMiddle.append((uniqueValues[i] + uniqueValues[i+1]) / 2)
                    
                    
                    
#                 ## UniqueMiddle currently contains the values to split over
#                 for j in uniqueMiddle:
#                     # compare values of X with values in uniqueMiddle
#                     if X 
                
                
                
#                 print(uniqueMiddle)
            
#             ##The feature is categorical
#             else:
                
#                 splitsX = []
#                 splitsy = []
                
#                 infoGain = startEntropy
#                 for value in uniqueValues:
                    
#                     ## Creating a list of X and y for the current split
#                     splitX = []
#                     splity = []
                    
#                     for sampleNumber in range(0, len(X)):
#                         if X[sampleNumber][featureNumber] == value:
#                             splitX.append(X[sampleNumber])
#                             splity.append(y[sampleNumber])
                            
                            
#                     splitsX.append(splitX)
#                     splitsy.append(splity)
                            
#                     entProbabilities = list(class_probabilities(splity).values())
#                     splitEntropy = entropy(entProbabilities)
                    
#                     infoGain =- (len(splity[0]) / len(y)) * splitEntropy
                    
#                 if infoGain > winningIG:
#                     winningIG = infoGain
#                     winningFeature = featureNumber
#                     winningXSplits = splitsX
#                     winningySplits = splitsy
                    
#                 print(uniqueValues)
#         return 
            
        
# #         for featIdx in range(n_features):
            
            
            
# #             featValues = dataset[:,featIdx]
# #             possThresh = np.unique(featValues)
            
            
# #             for threshold in possThresh:
                
# #                 #
# #                 dataLeft, dataRight = self.infoSplit(dataset, featIdx, threshold)
                
# #                 #
# #                 if len(dataLeft) > 0 and len(dataRight) > 0:
# #                     #
# #                     y, yLeft, yRight = dataset[:, -1], dataLeft[:, -1], dataRight[:, -1]
# #                     #
# #                     currentIG = self.infoGain(y, yLeft, yRight, "entropy")
                    
# #                     if currentIG > maxInfogain:
# #                         bestSplit["featureInd"] = featIdx
# #                         bestSplit["dataLeft"] = dataLeft
# #                         bestSplit["dataRight"] = dataRight
# #                         bestSplit["infoGain"] = currentIG
# #                         bestSplit["threshold"] = threshold
# #                         maxInfogain = currentIG
        
# #         return bestSplit
# #         #########################
        

    
#     def infoSplit(self, dataset, featIdx, threshold):
        
#         left = np.array([])
#         for j in dataset:
#             if j[featIdx]<=threshold:
#                 np.append(dataLeft, j[featIdx])
        
#         right = np.array([])
#         for r in dataset:
#             if r[featIdx]>threshold:
#                 np.append(dataRight, r[featIdx])
                
            
#         return dataLeft, dataRight
#         #########################
    
#     def infoGain(self, parent, lChild, rChild, mode="entropy"):
#         # IG = entropy(Parent) - [weighted average] * E(children)
        
#         pass
        
    
#     def predict(self, X):
        
#         print("Feature Number:", self.root.featureIndex)
        
        
        
    
# #         self.threshold = threshold
# #         self.left = left
# #         self.right = right
# #         self.info_gain = info_gain
# #         self.value = value   ## for leaf node calculation

# #         # the fololwing block of code handles lists/arrays containing multiple samples
# #         X = np.array(X)
# #         if len(X.shape) >= 2:
# #             return np.array([self.predict(x) for x in X])
        
# #         # in the following section, you can assume that X is a single n-dimensional sample:
# #         prediction = []
        
# #         for X in #######:
# #             if self.root.value != None: 
# #                 return self.root.value
# #             featureVals = X[self.root.featureIndex]
# #             if featureVals <= self.root.threshold:
# #                 np.append(prediction, X, self.root.left)
# #             else:
# #                 np.append(prediction, X, self.root.right)
            
# #         return most_common_class(prediction)


    
# class Node():
#     def __init__(self, featureIndex=None, threshold=None, left=None, right=None, info_gain=None, value=None):
                 
#         # for decision node
#         self.featureIndex = featureIndex
#         self.threshold = threshold
#         self.left = left
#         self.right = right
#         self.info_gain = info_gain
#         self.value = value   ## for leaf node calculation

#### TESTING CELLS:

The following cell will test if the classifier implemented model can fit to the [numerical dataset](#Numerical-data), and predict solutions for some of the testing samples (this is not model evaluation yet):

In [60]:
# Initialise the classifier, requiring at least 3 samples in each node
classifier = DecisionTree(min_samples_per_node = 3)
# Fit the classifier to the training data
classifier.fit(X_nd_train, y_nd_train)

# Predict the classification for some samples to the testing set
for (sample, prediction, truth) in zip(X_nd_test[3:10], classifier.predict(X_nd_test[3:10]), y_nd_test[3:10]):
    # Print the sample, predicted class and ground truth
    print("Sample: {}\n\tPredicted class: {}\n\tTrue class: {}".format(sample, prediction, truth))

Sample: [-2.45500e+00  7.00000e-02  2.34400e+00  2.60000e+01  3.21300e+00
  3.09993e+02]
	Predicted class: triclinic
	True class: orthorhombic
Sample: [-2.35200e+00  8.30000e-02  1.36700e+00  2.80000e+01  2.93500e+00
  3.57496e+02]
	Predicted class: monoclinic
	True class: orthorhombic
Sample: [-2.55300e+00  7.30000e-02  2.64900e+00  3.20000e+01  2.87700e+00
  3.73622e+02]
	Predicted class: triclinic
	True class: monoclinic
Sample: [-2.43900e+00  5.70000e-02  2.30000e-01  1.50000e+01  3.02400e+00
  1.77312e+02]
	Predicted class: triclinic
	True class: triclinic
Sample: [-2.88700e+00  4.00000e-02  3.14400e+00  5.20000e+01  2.69000e+00
  6.79101e+02]
	Predicted class: monoclinic
	True class: monoclinic
Sample: [-2.56400e+00  5.80000e-02  2.73000e+00  2.80000e+01  2.85600e+00
  3.60121e+02]
	Predicted class: orthorhombic
	True class: orthorhombic
Sample: [-2.91100e+00  6.40000e-02  3.07900e+00  3.40000e+01  2.63300e+00
  4.31422e+02]
	Predicted class: triclinic
	True class: triclinic


The following cell will check if the dataset is able to fit the model to the [dataset with mixed numerical and categorical values](#Categorical-data), and predict solutions for some of the testing samples (this is not model evaluation yet):

_**Warning:**_ Executing this cell may take a bit of time.

In [61]:
# # Initialise the classifier, requiring the decision tree to have no more than 5 levels
# classifier = DecisionTree(max_depth = 4)
# # Fit the classifier to the training data
# classifier.fit(X_cd_train, y_cd_train)

# # Predict the classification for some samples to the testing set
# for (sample, prediction, truth) in zip(X_cd_test[3:10], classifier.predict(X_cd_test[3:10]), y_cd_test[3:10]):
#     # Print the sample, predicted class and ground truth
#     print("Sample: {}\n\tPredicted class: {}\n\tTrue class: {}".format(sample, prediction, truth))

## Model evaluation and analysis

Executing all the cells from the previous section correctly only means that your implementation of [kNN classifier](#knn-Classifier) and [Decision Tree classifier](#Decision-Tree-classifier) runs on the provided data. This has still not provided any insight on how well those models represent the data.

In this section, you are instead required to calculate some evaluation metrics, and evaluate the produced models on the whole of the held-out data (test set) defined in the [Dataset](#Dataset) section.

### Task 4.1 - Evaluation
#### 10% marks

You are required to write a function that will calculate different evaluation metrics based on the ground truth labels and the predicted labels. Specifically, write a function:
- `evaluate(y_true, y_pred)`: evaluate classification results

    *Inputs:*<br>
    `y_true` : Ground truth labels for $n$ samples. This is either a list of $n$ elements, or a `numpy.array` of dimensions `(n,)`
    `y_pred` : Predicted labels for $n$ samples. This is either a list of $n$ elements, or a `numpy.array` of dimensions `(n,)`
    
    *Returns:*<br>
    A touple `(a, p, r, k)` consisting of the following values:
    - `a` : overall accuracy of the predictions
    - `p` : a dictionary containing a precision score for each class present in `y_true`
    - `r` : a dictionary containing a recall score for each class present in `y_true`
    - `k` : Cohen's Kappa metric

#### SOLUTION CELL:

In [68]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, cohen_kappa_score ## woudlnt work otherwise

def evaluate(y_true, y_pred):
    
    output = []
    acc = accuracy_score(y_true, y_pred)
    prc = precision_score(y_true, y_pred, average = None, labels = np.unique(y_true))
    rec = recall_score(y_true, y_pred, average = None, labels = np.unique(y_true))
    cKappa = cohen_kappa_score(y_true, y_pred)
    
    ## make dict of both metrics for append to output.
    precision = dict(zip(np.unique(y_true), prc))
    recall = dict(zip(np.unique(y_true), rec))
    
    ## append variables to output array.
    output = [acc, recall, precision, cKappa]
    
    return output 
    

#### TESTING CELL:

The following cell evaluates your implementation of [kNN classifier](#Task-2.2---kNN-classifier) and [Decision Tree classifier](#Task-3.2---Decision-Tree-classifier) on the numerical dataset defined in the [Dataset](#Dataset) section:

In [69]:
classifier_knn = knn(k=5)
classifier_knn.fit(X_nd_train, y_nd_train)

classifier_dt = DecisionTree(min_samples_per_node = 3)
classifier_dt.fit(X_nd_train, y_nd_train)

y_knn = classifier_knn.predict(X_nd_test)
y_dt = classifier_dt.predict(X_nd_test)

a_knn, p_knn, r_knn, k_knn = evaluate(y_nd_test, y_knn)
a_dt, p_dt, r_dt, k_dt = evaluate(y_nd_test, y_dt)

print("Evaluating numerical dataset, using kNN:")
print("Accuracy = {:.4f}, Cohen's Kappa = {:.4f}".format(a_knn, k_knn))
for key in p_knn.keys():
    print("\tClass {}: precision = {:.2f}, recall = {:.2f}".format(key, p_knn[key], r_knn[key]))

print("Evaluating numerical dataset, using DecisionTree:")
print("Accuracy = {:.4f}, Cohen's Kappa = {:.4f}".format(a_dt, k_dt))
for key in p_dt.keys():
    print("\tClass {}: precision = {:.2f}, recall = {:.2f}".format(key, p_dt[key], r_dt[key]))

Evaluating numerical dataset, using kNN:
Accuracy = 0.5833, Cohen's Kappa = 0.3651
	Class monoclinic: precision = 0.62, recall = 0.56
	Class orthorhombic: precision = 0.60, recall = 0.67
	Class triclinic: precision = 0.50, recall = 0.50
Evaluating numerical dataset, using DecisionTree:
Accuracy = 0.5000, Cohen's Kappa = 0.2480
	Class monoclinic: precision = 0.69, recall = 0.41
	Class orthorhombic: precision = 0.40, recall = 0.73
	Class triclinic: precision = 0.42, recall = 0.50


The following cell evaluates your implementation of [Decision Tree classifier](#Decision-Tree-classifier) on the [dataset containing a mixture of numerical and categorical data](#Categorical-data):

_**Warning:**_ Executing this cell may take a bit of time.

In [64]:
# classifier_categorical = DecisionTree(max_depth = 4)
# classifier_categorical.fit(X_cd_train, y_cd_train)

# y_cd_pred = classifier_categorical.predict(X_cd_test)
# a_cat, p_cat, r_cat, k_cat = evaluate(y_cd_test, y_cd_pred)

# print("Evaluating categorical dataset, using DecisionTree:")
# print("Accuracy = {:.4f}, Cohen's Kappa = {:.4f}".format(a_cat, k_cat))
# for key in p_cat.keys():
#     print("\tClass {}: precision = {:.2f}, recall = {:.2f}".format(key, p_cat[key], r_cat[key]))


### Task 4.2 - Analysis
#### 10% marks

Answer the following questions in the space provided below (marked as **YOUR ANSWERS**):
1. Explain the difference between the different performance metrics calculated for [Task 4.1](#Task-4.1---Evaluation). Which additional metrics could you propose to evaluate the performance of the defined models? Which of the calculated metrics would you chose to report, and why, for the analysis of the two [datasets](#Datasets) used in this assessment?

2. In case you were handling a _very large_ amount of data, which of the models do you expect to be larger (i.e. take up more memory)? Which of the models do you expect to train faster? Which of the models do you expect to reach a decision faster? Explain and justify your answers.

3. When handling numerical data, what is the role of normalisation? The numerical data used in this assessment was not normalised. If you were to normalise the data before training these classification models, would either of them change how they represent the data (and potentially return different results)?

4. For evaluating the performance of the classifiers implemented for this assignment, $80\%$ of the data was used for training the models, and $20\%$ for testing. What are the shortcomings of this evaluation strategy, and can you propose and describe a better one?

_Note:_ You may use the cell marked as **EXPERIMENTAL CELL** to write any code that could help you answer the above questions, or in general, to experiment with the code produced for this assessment.

#### EXPERIMENTAL CELL:

In [65]:
############ You may use this cell to write any code which may help you answers the questions above ##############
####### Your implementations from this cell will not be assessed, feel free to use it for experimentation ########

#### YOUR ANSWERS:

1. <font color='red'>In terms of Machine Learning evaluation, Accuracy depicts the rate of how often the model predicts correctly as opposed to not, Precision is the rate at which the model predicts in agreement of the proposal correctly against the incorrect votes. Recall is the rate at which correctly predicted values are compared to the total values that resulted in the true positive result. Cohen's Kappa evaluation metric is used to assess the correlation of agreement between two raters which can be displayed in a correlation matrix to gain better understanding of the models' predictions. 
In addition to these performance metrics, I would also employ the use of f-measure and ROC graphs to visualise accuracy based on true and false positive values. better understand how accurately decisions are made.
Cohen's kappa is the better performance metric to report from this evaluation because it thrives above others when the datasets in question have a varying number of classes. This metric is often used in fields such as medical diagnostics because of the vast array of false positive and negative results. Kappa is especially useful when comparing the performance  of multiple classifiers as it can provide a standardised measure between the classifier agreements.

</font> 
2. <font color='red'>In comparison to one another, a decison tree will take up more memory in total because it stores all of the splits, nodes and leaves in its memory which will obviously increase in correlation with the scale of the datasets. Whereas a KNN classifier doesn't need to store an entire tree structure with all decision points; it only needs the training data and distance metrics in order to function. Alternatively, a KNN classifier will be quicker to decide on smaller datasets because the computational logic behind it is far less cumbersome; a decision tree must traverse all nodes before making a decision. however, a decision tree does best with large datasets and can handle more complex data relationships, including comparisons between numerical and categorical data.

</font> 

3. <font color='red'>Normalisation in the sense of training a classifier is of paramount importance when in search of fair results; this process ensures that the data has a mean of 0 and a standard deviation of 1 so that the classifier doesn't favour this data over other entries just because of its scale. for example, if 2 separate data entries separately have a scale of 0-1 and 0-100, it will favour the latter because of this. 
Ultimately, normalisation removes any bias the classifier may have in processing the data.
if we were to normalise the data before training the classifier, we would have gotten more accurate results as the model would have less biased decision making, selecting best features more accurately. It provides a better comparison when comparing results with separate classifiers or on different datasets. It is important to note that normalised data doesn't affect the overall performance of a decision tree; although it will affect a KNN model as the features will have similar ranges making them easier to be detected.

</font> 
4. <font color='red'>With the 80% train 20% test approach, there is a prevalent possibility of underfitting as the training set is too big and the classifier might not well generalise new data; this will cause the classifier to process unseen data ineffectively in later classification. when the dataset is unbalanced an 80/20 split may be less effective due to class under representation, a more even split like 60/40 may be more appropriate for itself and high-dimensional data. 

For this evaluation, a metric that i would use instead of the ones stated would be K-fold cross-validation because it allows for a more precise and effective evaluation of a models' performance, especially more so on unseen data. It tests the models data by splitting into subsets of k partitions and repeating the process on all partitions to find the average result of this metric and reducing the risk of overfitting the data.</font> 